In [4]:
import os
import pandas as pd
import tensorflow as tf 
import numpy as np
import glob
from sklearn import metrics
from sklearn.metrics import classification_report


/Users/jc/code/ich-detection


/Users/jc/.venvs/bme548/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### RUN THIS CELL IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
if os.getcwd().split('/')[-1] == "evaluation":
    %cd ../..

# Import noise simulator
from scripts.utils import read_dcm

# Import model
from scripts.model import VGG

### IF RUNNING ON GOOGLE COLAB OR KAGGLE, RUN THE NEXT CELL

In [ ]:
# Clone the repository
if not os.path.exists("ICH-DETECTION"):
    % git clone https://github.com/jpscardoso97/ich-detection.git 
    % git checkout feat/repo-restructure

# Change to the repository directory
%cd ICH-DETECTION

# Import noise simulator
from scripts.utils import read_dcm

# Import model
from scripts.model import VGG

## Load Data

In [9]:
def get_train_and_test_data():
  train_df=pd.read_csv("data/RSNA_BME548/train/data.csv")
  test_df=pd.read_csv("data/RSNA_BME548/test/data.csv")

  train_files=glob.glob("data/RSNA_BME548/train/images/*.dcm")
  test_files=glob.glob("data/RSNA_BME548/test/images/*.dcm")

  train_label=[]
  test_label=[]

  train_img_array=[]
  test_img_array=[]

  for i in train_files:
    filename = os.path.basename(i)
    img=read_dcm(i)
    train_img_array.append(np.array(img))
    # Get value of column "any" for the row with filename as index
    train_label.append(train_df.loc[train_df['filename'] == filename]['any'].values[0])

  for i in test_files:
    filename = os.path.basename(i)
    img=read_dcm(i)
    test_img_array.append(np.array(img))
    # Get value of column "any" for the row with filename as index
    test_label.append(test_df.loc[test_df['filename'] == filename]['any'].values[0])

  X_train = np.array(train_img_array)/255
  X_train = np.expand_dims(X_train,3)
  y_train = np.array(train_label)

  X_test = np.array(test_img_array)/255
  X_test = np.expand_dims(X_test,3)
  y_test = np.array(test_label)
  
  return X_train, X_test, y_train, y_test

In [12]:
def get_model_accuracy(X_train, X_test, y_train, y_test):
  vgg = VGG(num_classes=2)
  vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  vgg.fit(X_train,y_train,epochs=5)

  y_pred=vgg.predict(X_test)

  y_pred_binary=[]
  for a in y_pred:
    y_pred_binary.append(np.argmax(a))

  y_pred_binary=np.array(y_pred_binary)

  accuracy = metrics.accuracy_score(y_test,y_pred_binary)

  print("Accuracy of model=",accuracy)

  print(classification_report(y_test, y_pred_binary, target_names=['0','1']))

  return accuracy

In [13]:
X_train, X_test, y_train, y_test = get_train_and_test_data()

In [ ]:
get_model_accuracy(X_train, X_test, y_train, y_test)